In [1]:
import os
from glob import glob
import numpy as np
import pandas as pd
import cv2
from skimage.feature import hog, local_binary_pattern
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.metrics import silhouette_score, davies_bouldin_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings("ignore")

DATA_DIR = os.path.expanduser(r"chest_xray") 
IMG_SIZE = (256, 256)
HIST_BINS = 256
HOG_PIXELS_PER_CELL = (8, 8)
HOG_CELLS_PER_BLOCK = (2, 2)
HOG_ORIENTATIONS = 9
LBP_P = 8
LBP_R = 1
LBP_N_BINS = 59
RANDOM_STATE = 42

PCA_N_COMPONENTS_FOR_CLUSTERING = 100   # reduce to this many principal components for clustering
PCA_N_COMPONENTS_FOR_TSNE = 50          # PCA before t-SNE (speeds up t-SNE)
TSNE_PERPLEXITY = 30

# KMeans selection
K_RANGE = range(2, 7)  # try K from 2..6
KMEANS_N_INIT = 10

# DBSCAN
DBSCAN_EPS = 2.5
DBSCAN_MINPTS = 5

# OUTPUT FILES
FEATURES_CSV = "xray_features_scaled.csv"
CLUSTERS_CSV = "xray_features_with_clusters.csv"
EVAL_CSV = "clustering_evaluation.csv"
OUT_DIR = "clustering_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

def find_images(root_dir, exts=('png','jpg','jpeg','bmp')):
    files = []
    for e in exts:
        files.extend(glob(os.path.join(root_dir, '**', f'*.{e}'), recursive=True))
    return sorted(files)

def read_preprocess(path, size=IMG_SIZE):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None:
        return None
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, size, interpolation=cv2.INTER_AREA)
    if img.dtype != np.uint8:
        img = cv2.normalize(img, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
    return img

def hist_feat(img, bins=HIST_BINS):
    hist = cv2.calcHist([img], [0], None, [bins], [0,256]).flatten()
    hist = hist / (hist.sum() + 1e-8)
    return hist

def hog_feat(img):
    img_f = img.astype('float32') / 255.0
    v = hog(img_f,
            orientations=HOG_ORIENTATIONS,
            pixels_per_cell=HOG_PIXELS_PER_CELL,
            cells_per_block=HOG_CELLS_PER_BLOCK,
            block_norm='L2-Hys',
            feature_vector=True)
    return v

def lbp_hist_feat(img, P=LBP_P, R=LBP_R, bins=LBP_N_BINS):
    lbp = local_binary_pattern(img, P, R, method='uniform')
    (hist, _) = np.histogram(lbp.ravel(), bins=np.arange(0, bins+1), range=(0, bins))
    hist = hist.astype('float32')
    hist = hist / (hist.sum() + 1e-8)
    return hist

def hu_moments_feat(img):
    m = cv2.moments(img.astype('float32'))
    hu = cv2.HuMoments(m).flatten()
    with np.errstate(divide='ignore'):
        hu_log = -np.sign(hu) * np.log10(np.abs(hu) + 1e-30)
    hu_log = np.nan_to_num(hu_log)
    return hu_log

def extract_label(path):
    parts = path.replace('\\','/').split('/')
    for tok in reversed(parts[:-1]):
        t = tok.upper()
        if 'NORMAL' in t:
            return 'NORMAL'
        if 'PNEUMONIA' in t:
            return 'PNEUMONIA'
    return 'UNKNOWN'

image_paths = find_images(DATA_DIR)
print(f"Found {len(image_paths)} images in {DATA_DIR}")

features = []
paths = []
true_labels = []
failed = 0
start = time.time()

for p in tqdm(image_paths, desc="Processing images"):
    img = read_preprocess(p)
    if img is None:
        failed += 1
        continue
    v_hist = hist_feat(img)
    v_hog = hog_feat(img)
    v_lbp = lbp_hist_feat(img)
    v_hu  = hu_moments_feat(img)
    v = np.concatenate([v_hist, v_hog, v_lbp, v_hu])
    features.append(v)
    paths.append(p)
    true_labels.append(extract_label(p))

features = np.array(features)
print(f"Loaded {features.shape[0]} feature vectors. Failed: {failed}")
print("Feature matrix shape:", features.shape)
print("Elapsed (feature extraction): {:.1f}s".format(time.time() - start))

scaler = StandardScaler()
X_scaled = scaler.fit_transform(features)
# Save scaled features with metadata
df_feat = pd.DataFrame(X_scaled)
df_feat['path'] = paths
df_feat['true_label'] = true_labels
df_feat.to_csv(os.path.join(OUT_DIR, FEATURES_CSV), index=False)
print("Saved scaled features at", os.path.join(OUT_DIR, FEATURES_CSV))

print("Running PCA to reduce to", PCA_N_COMPONENTS_FOR_CLUSTERING, "components...")
pca_cluster = PCA(n_components=PCA_N_COMPONENTS_FOR_CLUSTERING, random_state=RANDOM_STATE)
X_pca_cluster = pca_cluster.fit_transform(X_scaled)
print("Explained variance (sum):", pca_cluster.explained_variance_ratio_.sum())

inertias = []
sil_scores = []
ks = list(K_RANGE)
print("Running KMeans for K in", ks)
for k in ks:
    km = KMeans(n_clusters=k, n_init=KMEANS_N_INIT, random_state=RANDOM_STATE)
    lab = km.fit_predict(X_pca_cluster)
    inertias.append(km.inertia_)
    try:
        sil = silhouette_score(X_pca_cluster, lab)
    except:
        sil = np.nan
    sil_scores.append(sil)

# Save elbow and silhouette plots
plt.figure()
plt.plot(ks, inertias, marker='o')
plt.xlabel('K')
plt.ylabel('Inertia')
plt.title('Elbow: K vs Inertia')
plt.grid(True)
plt.savefig(os.path.join(OUT_DIR, 'k_elbow.png'), dpi=150)
plt.close()

plt.figure()
plt.plot(ks, sil_scores, marker='o')
plt.xlabel('K')
plt.ylabel('Silhouette Score')
plt.title('Silhouette: K vs Score')
plt.grid(True)
plt.savefig(os.path.join(OUT_DIR, 'k_silhouette.png'), dpi=150)
plt.close()

# Choose K: pick K with max silhouette (fallback to 2)
best_k_idx = int(np.nanargmax(sil_scores)) if any(~np.isnan(sil_scores)) else 0
best_k = ks[best_k_idx] if ks else 2
print("Best K (by silhouette) =", best_k)

# Run final KMeans with best_k
kmeans = KMeans(n_clusters=best_k, n_init=KMEANS_N_INIT, random_state=RANDOM_STATE)
labels_kmeans = kmeans.fit_predict(X_pca_cluster)

# ----------------------
# Agglomerative (on PCA space)
# ----------------------
agg = AgglomerativeClustering(n_clusters=best_k, linkage='ward')
labels_agg = agg.fit_predict(X_pca_cluster)

# ----------------------
# DBSCAN (on PCA space)
# ----------------------
db = DBSCAN(eps=DBSCAN_EPS, min_samples=DBSCAN_MINPTS, metric='euclidean')
labels_dbscan = db.fit_predict(X_pca_cluster)


def safe_metrics(X, labels):
    res = {'silhouette': None, 'davies_bouldin': None, 'n_clusters': None}
    lab_unique = set(labels)
    lab_non_noise = [l for l in lab_unique if l != -1]
    res['n_clusters'] = len(lab_non_noise) if -1 in lab_unique else len(lab_unique)
    try:
        if len(set(labels)) > 1:
            res['silhouette'] = float(silhouette_score(X, labels))
            res['davies_bouldin'] = float(davies_bouldin_score(X, labels))
    except Exception as e:
        pass
    return res

evals = []
eval_km = safe_metrics(X_pca_cluster, labels_kmeans)
eval_km.update({'method': 'kmeans', 'labels_array': labels_kmeans})
evals.append(eval_km)
eval_agg = safe_metrics(X_pca_cluster, labels_agg)
eval_agg.update({'method': 'agglomerative', 'labels_array': labels_agg})
evals.append(eval_agg)
eval_db = safe_metrics(X_pca_cluster, labels_dbscan)
eval_db.update({'method': 'dbscan', 'labels_array': labels_dbscan})
evals.append(eval_db)

eval_df = pd.DataFrame([{
    'method': e['method'],
    'n_clusters': e['n_clusters'],
    'silhouette': e['silhouette'],
    'davies_bouldin': e['davies_bouldin']
} for e in evals])
eval_df.to_csv(os.path.join(OUT_DIR, EVAL_CSV), index=False)
print("Saved clustering evaluation to", os.path.join(OUT_DIR, EVAL_CSV))
print(eval_df)

# ----------------------
# t-SNE visualization (PCA -> t-SNE for speed)
# ----------------------
print("Running PCA (for t-SNE) to", PCA_N_COMPONENTS_FOR_TSNE, "components then t-SNE (2D)...")
pca_tsne = PCA(n_components=PCA_N_COMPONENTS_FOR_TSNE, random_state=RANDOM_STATE)
X_pca_tsne = pca_tsne.fit_transform(X_scaled)

tsne = TSNE(n_components=2, perplexity=TSNE_PERPLEXITY, init='pca', random_state=RANDOM_STATE, learning_rate='auto')
X_tsne = tsne.fit_transform(X_pca_tsne)

# Plot function
def plot_labels_2d(coords, labels, title, fname):
    plt.figure(figsize=(8,6))
    uniq = np.unique(labels)
    for lab in uniq:
        mask = labels == lab
        label_name = 'noise' if lab == -1 else str(lab)
        plt.scatter(coords[mask,0], coords[mask,1], s=8, label=label_name, alpha=0.7)
    plt.legend(markerscale=2, bbox_to_anchor=(1.05,1), loc='upper left')
    plt.title(title)
    plt.tight_layout()
    plt.savefig(os.path.join(OUT_DIR, fname), dpi=150)
    plt.close()

plot_labels_2d(X_tsne, labels_kmeans, f"KMeans (K={best_k}) t-SNE", "kmeans_tsne.png")
plot_labels_2d(X_tsne, labels_agg, f"Agglomerative (K={best_k}) t-SNE", "agg_tsne.png")
plot_labels_2d(X_tsne, labels_dbscan, f"DBSCAN (eps={DBSCAN_EPS}, minpts={DBSCAN_MINPTS}) t-SNE", "dbscan_tsne.png")

# Plot true labels (if available)
if len(set(true_labels)) > 1:
    true_enc, uniques = pd.factorize(true_labels)
    plot_labels_2d(X_tsne, true_enc, "True labels (t-SNE)", "true_labels_tsne.png")

# ----------------------
# Save full CSV with cluster labels
# ----------------------
df_out = pd.DataFrame({
    'path': paths,
    'true_label': true_labels,
    'kmeans_label': labels_kmeans,
    'agg_label': labels_agg,
    'dbscan_label': labels_dbscan
})
# Append first few PCA components for convenience
for i in range(min(10, X_pca_cluster.shape[1])):
    df_out[f'pca_{i+1}'] = X_pca_cluster[:, i]

df_out.to_csv(os.path.join(OUT_DIR, CLUSTERS_CSV), index=False)
print("Saved clusters CSV to", os.path.join(OUT_DIR, CLUSTERS_CSV))

# ----------------------
# Optional: show confusion / purity vs true labels
# ----------------------
def cluster_purity(true_labels, cluster_labels):
    df = pd.DataFrame({'true': true_labels, 'cluster': cluster_labels})
    total = len(df)
    purity = 0.0
    for c in df['cluster'].unique():
        sub = df[df['cluster'] == c]
        if len(sub) == 0: continue
        top_count = sub['true'].value_counts().max()
        purity += top_count
    return purity / total

purity_km = cluster_purity(true_labels, labels_kmeans)
purity_agg = cluster_purity(true_labels, labels_agg)
purity_db = cluster_purity(true_labels, labels_dbscan)

pur_df = pd.DataFrame([{
    'method': 'kmeans', 'purity': purity_km
},{
    'method': 'agg', 'purity': purity_agg
},{
    'method': 'dbscan', 'purity': purity_db
}])
pur_df.to_csv(os.path.join(OUT_DIR, 'cluster_purity.csv'), index=False)
print("Saved cluster purity summary to", os.path.join(OUT_DIR, 'cluster_purity.csv'))
print(pur_df)

print("All outputs saved to directory:", os.path.abspath(OUT_DIR))
print("Done.")


Found 11712 images in C:\Users\BHARGAV NAIDU\Downloads\archive (2)\chest_xray


Processing images: 100%|█████████████████████████████████████████████████████████| 11712/11712 [09:58<00:00, 19.56it/s]


Loaded 11712 feature vectors. Failed: 0
Feature matrix shape: (11712, 34918)
Elapsed (feature extraction): 600.7s
Saved scaled features at clustering_outputs\xray_features_scaled.csv
Running PCA to reduce to 100 components...
Explained variance (sum): 0.3255538243200587
Running KMeans for K in [2, 3, 4, 5, 6]
Best K (by silhouette) = 2
Saved clustering evaluation to clustering_outputs\clustering_evaluation.csv
          method  n_clusters  silhouette  davies_bouldin
0         kmeans           2    0.083631        3.214753
1  agglomerative           2    0.075555        3.462432
2         dbscan           2    0.031891        0.861130
Running PCA (for t-SNE) to 50 components then t-SNE (2D)...
Saved clusters CSV to clustering_outputs\xray_features_with_clusters.csv
Saved cluster purity summary to clustering_outputs\cluster_purity.csv
   method    purity
0  kmeans  0.729679
1     agg  0.729679
2  dbscan  0.729679
All outputs saved to directory: C:\Users\BHARGAV NAIDU\clustering_outputs
D